## Assignment 5 
### Part 1
- This notebook analyses the differences between the sexes by age in Ireland.
- Looking into: Weighted mean age (by Sex) & The difference between the sexes by age.


In [224]:
import pandas as pd
import numpy as np

In [225]:
url = "https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/FY006A/CSV/1.0/en"
df = pd.read_csv(url)
df.head()

,STATISTIC,Statistic Label,TLIST(A1),CensusYear,C02199V02655,Sex,C02076V03371,Single Year of Age,C03789V04537,Administrative Counties,UNIT,VALUE
0,FY006AC01,Population,2022,2022,-,Both sexes,-,All ages,IE0,Ireland,Number,5149139
1,FY006AC01,Population,2022,2022,-,Both sexes,-,All ages,2ae19629-1492-13a3-e055-000000000001,Carlow County Council,Number,61968
2,FY006AC01,Population,2022,2022,-,Both sexes,-,All ages,2ae19629-1433-13a3-e055-000000000001,Dublin City Council,Number,592713
3,FY006AC01,Population,2022,2022,-,Both sexes,-,All ages,2ae19629-149f-13a3-e055-000000000001,Dún Laoghaire Rathdown County Council,Number,233860
4,FY006AC01,Population,2022,2022,-,Both sexes,-,All ages,2ae19629-14a0-13a3-e055-000000000001,Fingal County Council,Number,330506


In [226]:
headers = df.columns.tolist()
headers

['STATISTIC',
 'Statistic Label',
 'TLIST(A1)',
 'CensusYear',
 'C02199V02655',
 'Sex',
 'C02076V03371',
 'Single Year of Age',
 'C03789V04537',
 'Administrative Counties',
 'UNIT',
 'VALUE']

In [227]:
drop_col_list = ['STATISTIC', 'Statistic Label','TLIST(A1)','CensusYear','C02199V02655','C02076V03371','C03789V04537','UNIT']
df.drop(columns=drop_col_list, inplace=True)

df = df[df["Administrative Counties"] == "Ireland"]
df = df[df["Single Year of Age"] != "All ages"]
df['Single Year of Age'] = df['Single Year of Age'].str.replace('Under 1 year', '0')
df['Single Year of Age'] = df['Single Year of Age'].str.replace('\D', '', regex=True)

df['Single Year of Age']=df['Single Year of Age'].astype('int64')
df['VALUE']=df['VALUE'].astype('int64')
#print (df.head(3))
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 303 entries, 32 to 9760
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Sex                      303 non-null    object
 1   Single Year of Age       303 non-null    int64 
 2   Administrative Counties  303 non-null    object
 3   VALUE                    303 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 11.8+ KB


<>:7: SyntaxWarning: invalid escape sequence '\D'
<>:7: SyntaxWarning: invalid escape sequence '\D'
C:\Users\OrangH\AppData\Local\Temp\ipykernel_22648\4250953487.py:7: SyntaxWarning: invalid escape sequence '\D'
  df['Single Year of Age'] = df['Single Year of Age'].str.replace('\D', '', regex=True)


In [228]:
df_nat = pd.pivot_table(df, 'VALUE',"Sex","Single Year of Age")
print (df_nat.head(3))
# write out the entire file to local machine
df_nat.to_csv("population_for_analysis.csv")

Single Year of Age      0        1        2        3        4        5    \
Sex                                                                        
Both sexes          57796.0  56420.0  59210.0  60484.0  61505.0  64026.0   
Female              28186.0  27545.0  28974.0  29483.0  29819.0  31342.0   
Male                29610.0  28875.0  30236.0  31001.0  31686.0  32684.0   

Single Year of Age      6        7        8        9    ...     91      92   \
Sex                                                     ...                   
Both sexes          66714.0  69001.0  70733.0  72196.0  ...  5731.0  4106.0   
Female              32622.0  33572.0  34437.0  35227.0  ...  3781.0  2809.0   
Male                34092.0  35429.0  36296.0  36969.0  ...  1950.0  1297.0   

Single Year of Age     93      94      95      96     97     98     99     100  
Sex                                                                             
Both sexes          3257.0  2455.0  1826.0  1283.0  949.0  62

In [229]:
df_nat = pd.pivot_table(df, values='VALUE', index="Single Year of Age", columns="Sex" ,aggfunc='sum')

# Add a difference column (e.g., Male - Female)
df_nat['Male_minus_Female'] = (df_nat['Male'] - df_nat['Female']).abs()

print(df_nat.head(3))

# Write out the entire file to local machine
df_nat.to_csv("population_for_analysis.csv")


Sex                 Both sexes  Female   Male  Male_minus_Female
Single Year of Age                                              
0                        57796   28186  29610               1424
1                        56420   27545  28875               1330
2                        59210   28974  30236               1262


## Weighted mean age (by sex)

In [230]:
# Compute weighted mean ages
w_mean_male = np.average(df_nat.index, weights=df_nat["Male"])
w_mean_female = np.average(df_nat.index, weights=df_nat["Female"])

print(f"Weighted mean age (Male): {w_mean_male:.2f}")
print(f"Weighted mean age (Female): {w_mean_female:.2f}")


Weighted mean age (Male): 37.74
Weighted mean age (Female): 38.94
